### Find Word Images Padding Picture In Picture

In [1]:
import cv2
import PIL
from PIL import Image, ImageFilter, ImageOps
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

file_ext = 1000

In [3]:
#word_lemma_all_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Lemma Stem POS/Result/3-2-Word In Visual Genome Merge"

visual_genome_process_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-0-Visual Genome Process"

path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-5-Find Word Images Padding Picture In Picture"

#Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def image_padding_aspect_ratio(data_folder_name, data_path, out_folder_name, out_path, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9):
       '''
       image_padding_aspect_ratio(data_folder_name, data_path, out_folder_name, out_path, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9)\n
       data_folder_name that include image files, data_path that for data_folder_name, out_folder_name, out_path that for out_folder_name are path and folder in string(str).\n
       others are cv2 parameter and image aspect ratio parameter.\n
       ex.\n
       output_path = f"/home/kurubal/Documents/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/28 Words"\n
       image_path = f"/home/kurubal/Documents/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/28 Words"\n
       image_padding_aspect_ratio("Images_SV_Rename", image_path, "Images_SV", output_path, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9)
       '''
       error_file_list = []
       result_path = f"{out_path}/{out_folder_name}"
       data_folder_path = f"{data_path}/{data_folder_name}"
       Path(result_path).mkdir(parents=True, exist_ok=True)
       files=[f for f in os.listdir(data_folder_path) if isfile(join(data_folder_path,f))]   
       for i in range(len(files)):       
          pathfilename=data_folder_path+'/'+files[i]
          #filename_without_ext = os.path.splitext(files[i])[0]
          #text = f"{text}"
          if isfile(f"{result_path}/{files[i]}"):
                 pass
          else:
                 try:
                        img = cv2.imread(f"{pathfilename}")
                        old_image_height, old_image_width, channels = img.shape  # 2718  1988

                        #aspect_ratio_width = 16
                        #aspect_ratio_height = 9

                        if (old_image_height/aspect_ratio_height) > (old_image_width/aspect_ratio_width):
                               new_image_width = int((old_image_height/aspect_ratio_height)*aspect_ratio_width)
                               new_image_height = int(old_image_height)
                        elif (old_image_height/aspect_ratio_height) < (old_image_width/aspect_ratio_width):
                               new_image_width = int(old_image_width)
                               new_image_height = int((old_image_width/aspect_ratio_width)*aspect_ratio_height)
                        elif (old_image_height/aspect_ratio_height) == (old_image_width/aspect_ratio_width):
                               new_image_width = int((old_image_height/aspect_ratio_height)*aspect_ratio_width)
                               new_image_height = int(old_image_height)
                        else:
                               new_image_width = old_image_width
                               new_image_height = old_image_height

                        color = padding_color
                        result = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)

                        # compute center offset
                        x_center = (new_image_width - old_image_width) // 2
                        y_center = (new_image_height - old_image_height) // 2

                        # copy img image into center of result image
                        result[y_center:y_center+old_image_height, x_center:x_center+old_image_width] = img            

                        cv2.imwrite(f"{result_path}/{files[i]}", result)
                 except:
                        print("Oops!", sys.exc_info()[0], "occurred.")
                        error_file_list.append(files[i])
                        
       return error_file_list

In [5]:
def image_padding_aspect_ratio_standalone(image_path_file, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9, overwrite = False):
       '''
       image_padding_aspect_ratio_standalone(image_path_file, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9, overwrite = False)\n
       image_path_file that is only one image file and result image save in same path. If overwrite True, result image will overwrite to main file, otherwise result\n
       image save with padded extention. others are cv2 parameter and image aspect ratio parameter.\n
       ex.\n
       image_path_file = "/home/kurubal/Downloads/2333869.jpg"\n
       image_padding_aspect_ratio_standalone(image_path_file, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9, overwrite = False)
       '''
       error_file_list = []
       size = (1600, 900)  # image new size (width, height)
       # path process
       path_split_list = os.path.split(image_path_file)
       main_path = path_split_list[0]  # part of before last one
       file = path_split_list[1]  # last file or folder
       filename_without_ext = os.path.splitext(file)[0]  # file without .jpg
       filename_ext = os.path.splitext(file)[1]  # file extention as .jpg
       # file process
       if isfile(f"{main_path}/{filename_without_ext}_padded{filename_ext}"):
              pass
       else:
              try:
                     img = cv2.imread(f"{image_path_file}")
                     old_image_height, old_image_width, channels = img.shape 

                     if (old_image_height/aspect_ratio_height) > (old_image_width/aspect_ratio_width):
                            new_image_width = int((old_image_height/aspect_ratio_height)*aspect_ratio_width)
                            new_image_height = int(old_image_height)
                     elif (old_image_height/aspect_ratio_height) < (old_image_width/aspect_ratio_width):
                            new_image_width = int(old_image_width)
                            new_image_height = int((old_image_width/aspect_ratio_width)*aspect_ratio_height)
                     elif (old_image_height/aspect_ratio_height) == (old_image_width/aspect_ratio_width):
                            new_image_width = int((old_image_height/aspect_ratio_height)*aspect_ratio_width)
                            new_image_height = int(old_image_height)
                     else:
                            new_image_width = old_image_width
                            new_image_height = old_image_height

                     color = padding_color
                     result = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)

                     # compute center offset
                     x_center = (new_image_width - old_image_width) // 2
                     y_center = (new_image_height - old_image_height) // 2

                     # copy img image into center of result image
                     result[y_center:y_center+old_image_height, x_center:x_center+old_image_width] = img

                     # image resize
                     result_resize = cv2.resize(result, size, interpolation = cv2.INTER_AREA)

                     if overwrite:
                            cv2.imwrite(f"{main_path}/{filename_without_ext}{filename_ext}", result_resize)  # for overwrite
                     else:
                            cv2.imwrite(f"{main_path}/{filename_without_ext}_padded{filename_ext}", result_resize)
              except:
                     print("Oops!", sys.exc_info()[0], "occurred.")
                     error_file_list.append(image_path_file)
                        
       return error_file_list

In [6]:
def picture_in_picture(main_image_file_path, add_image_file_path, add_pic_size=8, add_pic_pos=["top","left"], add_pic_margin=10, border_color="red"):
    '''picture_in_picture(main_image_file_path, add_image_file_path, add_pic_size=8, add_pic_pos=["top","left"], add_pic_margin=10, border_color="red")\n
    main_image_file_path is master image file path or result of image file, add_image_file_path is slave image that use for paste on master image.\n
    add_pic_size is slave image size that is master_image_width(1600px)/add_pic_size(8)=200px. add_pic_pos refers to slave image position on master image.\n
    add_pic_margin is slave image shift from edges. border_color is slave image border color for highlight.\n
    ex.\n
    picture_in_picture("gibi.jpg", "Group 36.jpg", add_pic_pos = ["bottom","left"], add_pic_margin = 10, border_color = "red")
    '''
    # main image
    try:  
        image_main = Image.open(f"{main_image_file_path}")  # 1600*900 px, image file path
    except:
        image_main = main_image_file_path  # picture of result from RAM
        
    width_main, height_main = image_main.size

    # additional image
    image_add = Image.open(f"{add_image_file_path}")  # square image
    height_add_edit = int(width_main/add_pic_size)  # 1600/8 = 200 px
    width_add_edit = int(width_main/add_pic_size)
    new_size_add = (width_add_edit, height_add_edit)  # (height_add_edit ,width_add_edit) ??
    image_add_resize = image_add.resize(new_size_add)
    # border
    border_thickness = (3, 3, 3, 3)
    image_add_resize_border = ImageOps.expand(image_add_resize, border=border_thickness, fill=border_color)
    width_add, height_add = image_add_resize_border.size

    if (add_pic_pos[0] == "top") and (add_pic_pos[1] == "left"):
        image_main.paste(image_add_resize_border, (add_pic_margin, add_pic_margin)) # x_koor, y_koor
    elif (add_pic_pos[0] == "top") and (add_pic_pos[1] == "right"):
        image_main.paste(image_add_resize_border, ((width_main-width_add-add_pic_margin), add_pic_margin))
    elif (add_pic_pos[0] == "bottom") and (add_pic_pos[1] == "left"):
        image_main.paste(image_add_resize_border, (add_pic_margin, (height_main-height_add-add_pic_margin)))
    elif (add_pic_pos[0] == "bottom") and (add_pic_pos[1] == "right"):
        image_main.paste(image_add_resize_border, ((width_main-width_add-add_pic_margin), (height_main-height_add-add_pic_margin)))
    elif (add_pic_pos[0] == "center") and (add_pic_pos[1] == "center"):
        image_main.paste(image_add_resize_border, ((int(width_main/2)-int(width_add/2)), (int(height_main/2)-int(height_add/2))))
    elif (add_pic_pos[0] == "left") and (add_pic_pos[1] == "center"):
        image_main.paste(image_add_resize_border, (add_pic_margin, (int(height_main/2)-int(height_add/2))))
    elif (add_pic_pos[0] == "right") and (add_pic_pos[1] == "center"):
       image_main.paste(image_add_resize_border, (width_main-width_add-add_pic_margin, (int(height_main/2)-int(height_add/2))))
    elif (add_pic_pos[0] == "top") and (add_pic_pos[1] == "center"):
        image_main.paste(image_add_resize_border, ((int(width_main/2)-int(width_add/2)), add_pic_margin))
    elif (add_pic_pos[0] == "bottom") and (add_pic_pos[1] == "center"):
        image_main.paste(image_add_resize_border, ((int(width_main/2)-int(width_add/2)), (height_main-height_add-add_pic_margin)))
    else:
        pass

    return image_main

#### Pronouns Image Padding Aspect Ratio

In [7]:
pron_output_path = f"/home/kurubal/Downloads"

Path(pron_output_path).mkdir(parents=True, exist_ok=True)

In [8]:
pron_image_path = f"/home/kurubal/Downloads"

In [9]:
#image_padding_aspect_ratio("Pronouns", pron_image_path, "Pronouns_Padding", pron_output_path, padding_color = (255,255,255), aspect_ratio_width = 1, aspect_ratio_height = 1)  # /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/Pronouns_Padding 

#### Select Image Padding

In [10]:
select_image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/{file_ext} Words/Image_Genome"

In [11]:
image_main_folder_list = os.listdir(f"{select_image_path}")
image_main_folder_list

['ALL_OTHER_FOLDER',
 'ALL_VERB_FOLDER',
 'ALL_ADJ_ADV_PRON_FOLDER',
 'MANUAL',
 'ALL_NOUN_FOLDER',
 'Backup Orj (No Padding)']

In [12]:
other_file_list = glob.glob(f"{select_image_path}/ALL_OTHER_FOLDER/*/*")  # other .jpg files
#other_file_list

In [13]:
verb_file_list = glob.glob(f"{select_image_path}/ALL_VERB_FOLDER/*/*/*")  # verb .jpg files
#verb_file_list

In [14]:
adj_adv_pros_file_list = glob.glob(f"{select_image_path}/ALL_ADJ_ADV_PRON_FOLDER/*/*/*")  # adj adv pros .jpg files
#adj_adv_pros_file_list

In [15]:
noun_file_list = glob.glob(f"{select_image_path}/ALL_NOUN_FOLDER/*/*/*")  # noun .jpg files
#noun_file_list

In [16]:
#for image_path_file in other_file_list:
#    image_padding_aspect_ratio_standalone(image_path_file, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9, overwrite = True)

In [17]:
#for image_path_file in verb_file_list:
#    image_padding_aspect_ratio_standalone(image_path_file, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9, overwrite = True)

In [18]:
#for image_path_file in adj_adv_pros_file_list:
#    image_padding_aspect_ratio_standalone(image_path_file, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9, overwrite = True)

In [19]:
#for image_path_file in noun_file_list:
#    image_padding_aspect_ratio_standalone(image_path_file, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9, overwrite = True)

#### Visual Genome Word Lemma All Category Data

##### Path And Folder

In [20]:
# Main image path (mandatory input data) 
main_image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/{file_ext} Words/Image_Genome"

In [21]:
image_main_folder_list = os.listdir(f"{main_image_path}")
image_main_folder_list

['ALL_OTHER_FOLDER',
 'ALL_VERB_FOLDER',
 'ALL_ADJ_ADV_PRON_FOLDER',
 'MANUAL',
 'ALL_NOUN_FOLDER',
 'Backup Orj (No Padding)']

In [22]:
# Additional image path folder (mandatory input data)
add_image_path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/Pronouns_Padding"

In [23]:
# Output result path folder
#output_path_folder = "/home/kurubal/Downloads/Genome_Picture_In_Picture"
output_path_folder = "//media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/1000 Words/Genome_Picture_In_Picture"

##### Main Excel File

In [24]:
#df_word_prefix_suffix_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Lemma Stem POS/Result/2-1-Word Prefix Suffix Analysis/{lang_folder.capitalize()}_{file_ext}_Word_Person_Case_Tense_Plural_Mean_Manuel_Modify.xlsx")  # modify main file
df_word_prefix_suffix_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/2-1-Word Prefix Suffix Analysis/{lang_folder.capitalize()}_{file_ext}_Word_Person_Case_Tense_Plural_Mean_Manuel.xlsx")  # main file
df_word_prefix_suffix_file

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
0,bir,NUM,NaN,bir,bir,bir,a,a,18835735,+,NaN,NaN,NaN,NaN,NaN,NaN
1,bir,ADV,NaN,biraz,biraz,bir,a little,a little,1269641,+az,NaN,NaN,NaN,NaN,NaN,NaN
2,bir,PRON,NaN,biri,biri,bir,somebody,somebody,837400,+i,NaN,NaN,belirtme,NaN,NaN,NaN
3,bir,ADV,NaN,birlikte,birlikte,bir,together,together,409940,+likte,NaN,NaN,NaN,NaN,NaN,NaN
4,bir,NOUN,NaN,birini,biri,bir,one,somebody,259916,+ini,NaN,NaN,belirtme,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,veriyor,VERB,NaN,veriyorum,ver,ver,i give,give,126290,+um,NaN,1.tekil,NaN,NaN,NaN,NaN
1620,veriyor,VERB,NaN,veriyor,ver,ver,giving,give,68163,+,şimdiki,NaN,NaN,NaN,NaN,NaN
1621,okul,NOUN,NaN,okul,okul,oku,school,school,68160,+,NaN,NaN,NaN,NaN,NaN,NaN
1622,suçlu,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124,+,NaN,3.tekil,belirtme,NaN,NaN,NaN


In [25]:
word_list = df_word_prefix_suffix_file["word"].unique()
#word_list

In [26]:
#selected_word = ['acil','affedersin','affedersiniz','ayrıca','dur','hadi','ister','işte','kaç','olarak','pardon','tahmin','takip','terk','umurumda','var','yazık','özür']  # not_found_result_in_genome
#df_word_prefix_suffix_file = df_word_prefix_suffix_file[df_word_prefix_suffix_file["word"].isin(selected_word)]  # for test
#df_word_prefix_suffix_file

In [27]:
df_word_prefix_suffix_file["word"].nunique()

1000

In [28]:
tense_image_name = list(df_word_prefix_suffix_file["tense"].dropna().unique())
person_image_name = list(df_word_prefix_suffix_file["person"].dropna().unique())
case_image_name = list(df_word_prefix_suffix_file["case"].dropna().unique())
plural_image_name = list(df_word_prefix_suffix_file["plural"].dropna().unique())
mean_image_name = list(df_word_prefix_suffix_file["mean"].dropna().unique())
other_image_name = list(df_word_prefix_suffix_file["other"].dropna().unique())
POS2_image_name = list(df_word_prefix_suffix_file["POS2"].dropna().unique())

In [29]:
add_image_list = tense_image_name + person_image_name + case_image_name + plural_image_name + mean_image_name + other_image_name + POS2_image_name
add_image_list
# Note: This add image list must be image file name as belirtme.jpg, yönelme.jpg etc. for next step is picture in picture

['geçmiş',
 'şimdiki',
 'gelecek',
 '3.tekil',
 '2.tekil',
 '1.tekil',
 '3.çoğul',
 '1.çoğul',
 '2.çoğul',
 'belirtme',
 'yönelme',
 'ayrılma',
 'bulunma',
 'çoğul',
 'olumsuz',
 'imperative',
 'to be',
 'Q',
 'NEG',
 'GRE',
 'TIME']

In [30]:
# main image and additional image picture in  picture
for row_num in range(len(df_word_prefix_suffix_file)):
    # main file result
    df_var = df_word_prefix_suffix_file.iloc[[row_num],]
    word = "".join(df_var["word"])
    try:
        tense = "".join(df_var["tense"])
        tense_file = glob.glob(f"{add_image_path_folder}/{tense}.jpg")
    except:
        tense_file = []
    try:
        person = "".join(df_var["person"])
        person_file = glob.glob(f"{add_image_path_folder}/{person}.jpg")
    except:
        person_file = []
    try:
        case = "".join(df_var["case"])
        case_file = glob.glob(f"{add_image_path_folder}/{case}.jpg")
    except:
        case_file = []
    try:
        plural = "".join(df_var["plural"])
        plural_file = glob.glob(f"{add_image_path_folder}/{plural}.jpg")
    except:
        plural_file = []
    try:
        mean = "".join(df_var["mean"])
        mean_file = glob.glob(f"{add_image_path_folder}/{mean}.jpg")
    except:
        mean_file = []
    try:
        other = "".join(df_var["other"])
        other_file = glob.glob(f"{add_image_path_folder}/{other}.jpg")
    except:
        other_file = []
    try:
        pos = "".join(df_var["POS2"])
        pos_file = glob.glob(f"{add_image_path_folder}/{pos}.jpg")
    except:
        pos_file = []

    add_image_file_list = tense_file + person_file + case_file + plural_file + mean_file + other_file + pos_file
    
    # word search in word folders
    file_list1 = glob.glob(f"{main_image_path}/ALL_VERB_FOLDER/*/{word}/*")
    file_list2 = glob.glob(f"{main_image_path}/ALL_NOUN_FOLDER/*/{word}/*")
    file_list3 = glob.glob(f"{main_image_path}/ALL_ADJ_ADV_PRON_FOLDER/*/{word}/*")
    file_list4 = glob.glob(f"{main_image_path}/ALL_OTHER_FOLDER/{word}/*")
    file_list5 = glob.glob(f"{main_image_path}/MANUAL/{word}/*")

    main_image_file_list = file_list1 + file_list2 + file_list3 + file_list4 + file_list5

    # create folder
    if len(main_image_file_list) == 0:
        pass
    else:
        folder_name = f"{word}"
        Path(f"{output_path_folder}/{folder_name}").mkdir(parents=True, exist_ok=True)

    # main image and additional image picture_in_picture
    main_num = 1
    for main_image_path_file in main_image_file_list:  # Note: when main_image_file_list is empty, process will pass automatically. 
        main_image = Image.open(f"{main_image_path_file}") 
        add_num = 1
        for add_image in add_image_file_list:
            if add_num == 1:
                main_image = picture_in_picture(main_image, f"{add_image}", add_pic_pos = ["top","left"], add_pic_margin = 10, border_color = "red")
            elif add_num == 2:
                main_image = picture_in_picture(main_image, f"{add_image}", add_pic_pos = ["top","right"], add_pic_margin = 10, border_color = "red")
            elif add_num == 3:
                main_image = picture_in_picture(main_image, f"{add_image}", add_pic_pos = ["bottom","left"], add_pic_margin = 10, border_color = "red")
            elif add_num == 4:
                main_image = picture_in_picture(main_image, f"{add_image}", add_pic_pos = ["bottom","right"], add_pic_margin = 10, border_color = "red")
            else:
                pass
            add_num += 1

        if isfile(f"{output_path_folder}/{word}/{word}{main_num}.jpg"):
            pass
        else:
            main_image.save(f"{output_path_folder}/{word}/{word}{main_num}.jpg")

        main_num += 1

##### Image's Not Found In Genome

In [38]:
path_folder_list = glob.glob(f"{output_path_folder}/*")
path_folder_list

['//media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/1000 Words/Genome_Picture_In_Picture/bir',
 '//media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/1000 Words/Genome_Picture_In_Picture/biraz',
 '//media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/1000 Words/Genome_Picture_In_Picture/biri',
 '//media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/1000 Words/Genome_Picture_In_Picture/birlikte',
 '//media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/1000 Words/Genome_Picture_In_Picture/birini',
 '//media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/1000 Words/Genome_Picture_In_Picture/birisi',
 '//media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/1000 Words/Genome_Picture_In_Picture/birinin',
 '//media/kurubal/SSD/Data Sci

In [39]:
word_folder_list = []
for path_folder in path_folder_list:
    split_path = os.path.split(path_folder)
    folder_var = split_path[1]  # last path name
    word_folder_list.append(folder_var)

In [40]:
word_set = set(word_list)
word_folder_set = set(word_folder_list)

In [41]:
not_found_result_in_genome = word_set.difference(word_folder_set)
not_found_result_in_genome

{'acil',
 'affedersin',
 'affedersiniz',
 'ayrıca',
 'dur',
 'hadi',
 'ister',
 'işte',
 'kaç',
 'olarak',
 'pardon',
 'tahmin',
 'takip',
 'terk',
 'umurumda',
 'var',
 'yazık',
 'özür'}

#### Pronouns Image New Size (For All Prons Image Same Size)

In [10]:
image_path_file = "/home/kurubal/Downloads/Pronouns_Padding/imperative.jpg"

In [11]:
def image_padding_aspect_ratio_standalone(image_path_file, padding_color = (255,255,255), aspect_ratio_width = 16, aspect_ratio_height = 9, overwrite = False):
       '''
       image_padding_aspect_ratio_standalone(image_path_file, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9, overwrite = False)\n
       image_path_file that is only one image file and result image save in same path. If overwrite True, result image will overwrite to main file, otherwise result\n
       image save with padded extention. others are cv2 parameter and image aspect ratio parameter.\n
       ex.\n
       image_path_file = "/home/kurubal/Downloads/2333869.jpg"\n
       image_padding_aspect_ratio_standalone(image_path_file, padding_color = (0,0,0), aspect_ratio_width = 16, aspect_ratio_height = 9, overwrite = False)
       '''
       error_file_list = []
       size = (200, 200)  # image new size (width, height)
       # path process
       path_split_list = os.path.split(image_path_file)
       main_path = path_split_list[0]  # part of before last one
       file = path_split_list[1]  # last file or folder
       filename_without_ext = os.path.splitext(file)[0]  # file without .jpg
       filename_ext = os.path.splitext(file)[1]  # file extention as .jpg
       # file process
       if isfile(f"{main_path}/{filename_without_ext}_padded{filename_ext}"):
              pass
       else:
              try:
                     img = cv2.imread(f"{image_path_file}")
                     old_image_height, old_image_width, channels = img.shape 

                     if (old_image_height/aspect_ratio_height) > (old_image_width/aspect_ratio_width):
                            new_image_width = int((old_image_height/aspect_ratio_height)*aspect_ratio_width)
                            new_image_height = int(old_image_height)
                     elif (old_image_height/aspect_ratio_height) < (old_image_width/aspect_ratio_width):
                            new_image_width = int(old_image_width)
                            new_image_height = int((old_image_width/aspect_ratio_width)*aspect_ratio_height)
                     elif (old_image_height/aspect_ratio_height) == (old_image_width/aspect_ratio_width):
                            new_image_width = int((old_image_height/aspect_ratio_height)*aspect_ratio_width)
                            new_image_height = int(old_image_height)
                     else:
                            new_image_width = old_image_width
                            new_image_height = old_image_height

                     color = padding_color
                     result = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)

                     # compute center offset
                     x_center = (new_image_width - old_image_width) // 2
                     y_center = (new_image_height - old_image_height) // 2

                     # copy img image into center of result image
                     result[y_center:y_center+old_image_height, x_center:x_center+old_image_width] = img

                     # image resize
                     result_resize = cv2.resize(result, size, interpolation = cv2.INTER_AREA)

                     if overwrite:
                            cv2.imwrite(f"{main_path}/{filename_without_ext}{filename_ext}", result_resize)  # for overwrite
                     else:
                            cv2.imwrite(f"{main_path}/{filename_without_ext}_padded{filename_ext}", result_resize)
              except:
                     print("Oops!", sys.exc_info()[0], "occurred.")
                     error_file_list.append(image_path_file)
                        
       return error_file_list

In [12]:
#image_padding_aspect_ratio_standalone(image_path_file, padding_color = (255,255,255), aspect_ratio_width = 1, aspect_ratio_height = 1)  # 200*200px

[]